In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import matplotlib.pyplot as plt
import numpy as np
import cv2
import random
import os

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

## Enregistrement dataset

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("HomerComp_test", {}, "./cocosplit/test.json", "./data/images/")
register_coco_instances("HomerComp_val", {}, ".//cocosplit/val.json", "./data/images/")
register_coco_instances("HomerComp_train", {}, "./cocosplit/train.json", "./data/images/")

In [ ]:
competition_metadata_test = MetadataCatalog.get("HomerComp_test")
dataset_dicts_test = DatasetCatalog.get("HomerComp_test")
competition_metadata_val = MetadataCatalog.get("HomerComp_val")
dataset_dicts_val = DatasetCatalog.get("HomerComp_val")
competition_metadata_train = MetadataCatalog.get("HomerComp_train")
dataset_dicts_train = DatasetCatalog.get("HomerComp_train")

## Visualisation dataset

In [ ]:
for i, d in enumerate(random.sample(dataset_dicts_train, 3)):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=competition_metadata_train, scale=0.8)
    vis = visualizer.draw_dataset_dict(d)
    cv2.imwrite(f'train_{i}.jpg', vis.get_image()[:, :, ::-1])

## Chargement / initialisation modèle

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger

In [ ]:
cfg = get_cfg()

In [ ]:
cfg.merge_from_file("./detectron2_repo/configs/COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")
cfg.DATASETS.TRAIN = ("HomerComp_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 1
cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-50.pkl"
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
cfg.SOLVER.BASE_LR = 0.005
cfg.SOLVER.MOMENTUM = 0.9
cfg.SOLVER.WEIGHT_DECAY = 0.0001
cfg.SEED = 1212
cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.SOLVER.MAX_ITER = 100000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 227

## Entraînement modèle

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

## Inférence

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_0003499.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.DATASETS.TEST = ("HomerComp_test", )
predictor = DefaultPredictor(cfg)

## Visualisation résultats

In [ ]:
from detectron2.utils.visualizer import ColorMode

for i, d in enumerate(random.sample(dataset_dicts_test, 25)):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=competition_metadata_test, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imwrite(f'result_{i}.jpg', v.get_image()[:, :, ::-1])